In [1]:
import torch
import numpy as np
import json

class PointSequenceDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        with open(data) as f:
            self.data = json.load(f)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq = torch.tensor(self.data[idx]["sequence"])
        cls = torch.tensor(self.data[idx]["intersect"]).long()
        return seq, cls

dataset = PointSequenceDataset("./lecture 4/train_dataset.json")
print(dataset[0])

(tensor([[0.9160, 0.2257],
        [1.2735, 0.2941],
        [1.1046, 0.5130],
        [1.3538, 0.2501],
        [1.4257, 0.2829],
        [1.3582, 0.7018],
        [1.5891, 0.5459],
        [1.6116, 0.6819],
        [1.8069, 0.2731],
        [2.1430, 0.1446],
        [2.3039, 0.3248]]), tensor(0))


In [9]:
class LSTMModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers=3, batch_first=True)
        self.ff = torch.nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.ff(out[:, -1, :])
        return out
        
dl = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
model = LSTMModel(2, 10, 2)

serie, cls = next(iter(dl))
print(serie.shape, cls.shape)
out = model(serie)
print(out.shape)

torch.Size([32, 11, 2]) torch.Size([32])
torch.Size([32, 2])


In [10]:
train_ds = PointSequenceDataset("./lecture 4/train_dataset.json")
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)
model = LSTMModel(2, 300, 2)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

model = model.to("cuda")

for epoch in range(20):
    for serie, cls in train_dl:
        serie = serie.to("cuda")
        cls = cls.to("cuda")
        
        out = model(serie)
        loss = loss_fn(out, cls)

        acc = (out.argmax(dim=1) == cls).float().mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch} Loss: {loss.item()} Acc: {acc.item()}")

Epoch 0 Loss: 0.6958112120628357 Acc: 0.40625
Epoch 0 Loss: 0.6965004205703735 Acc: 0.375
Epoch 0 Loss: 0.6932072043418884 Acc: 0.5
Epoch 0 Loss: 0.6968709826469421 Acc: 0.34375
Epoch 0 Loss: 0.6918213367462158 Acc: 0.5625
Epoch 0 Loss: 0.6945527195930481 Acc: 0.4375
Epoch 0 Loss: 0.6925787329673767 Acc: 0.53125
Epoch 0 Loss: 0.6912997364997864 Acc: 0.59375
Epoch 0 Loss: 0.6925349235534668 Acc: 0.53125
Epoch 0 Loss: 0.6949524879455566 Acc: 0.40625
Epoch 0 Loss: 0.6908642053604126 Acc: 0.625
Epoch 0 Loss: 0.6919419169425964 Acc: 0.5625
Epoch 0 Loss: 0.6949024200439453 Acc: 0.40625
Epoch 0 Loss: 0.6960737109184265 Acc: 0.34375
Epoch 0 Loss: 0.6948371529579163 Acc: 0.40625
Epoch 0 Loss: 0.6936918497085571 Acc: 0.46875
Epoch 0 Loss: 0.6916356682777405 Acc: 0.59375
Epoch 0 Loss: 0.6925883889198303 Acc: 0.53125
Epoch 0 Loss: 0.6916000843048096 Acc: 0.59375
Epoch 0 Loss: 0.696296751499176 Acc: 0.3125
Epoch 0 Loss: 0.6926061511039734 Acc: 0.53125
Epoch 0 Loss: 0.6930648684501648 Acc: 0.5
Epoch

In [4]:
test_ds = PointSequenceDataset("./lecture 4/test_dataset.json")
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=32, shuffle=True)

preds = []
actuals = []
for serie, cls in test_dl:
    serie = serie.to("cuda")
    cls = cls.to("cuda")
    
    out = model(serie)

    preds.append(out.argmax(dim=1).cpu().numpy())
    actuals.append(cls.cpu().numpy())

preds = np.concatenate(preds)
actuals = np.concatenate(actuals)

print((preds == actuals).mean())

0.667
